In [3]:
import pandas as pd


In [4]:
df=pd.read_csv('most_active_subreddits.csv')

In [65]:
node_data = df[['user_id', 'subreddit']]

In [6]:
user_ids = node_data['user_id'].tolist()

In [7]:
user_to_idx = {user_id: idx for idx, user_id in enumerate(user_ids)}

In [8]:
edges = []

In [9]:
folder_path ='social_graph_data'

In [10]:
import os

In [11]:
import torch

In [12]:
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        with open(os.path.join(folder_path, filename), 'r') as file:
            for line in file:
                parts = line.strip().split(';')
                user_id = parts[0]
                replied_user = parts[3]

                if user_id in user_to_idx and replied_user in user_to_idx:
                    edges.append((user_to_idx[user_id], user_to_idx[replied_user]))

In [13]:
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

In [30]:
label_mapping = {'r/politics': 0, 'r/NoNewNormal': 2, 'r/Sports': 3, 'r/Coronavirus':4, 'r/LockdownSkepticism': 5, 'r/JoeBiden':6, 'r/MensRights': 7, 'r/DebateVaccines':8,'r/climateskeptics':9,"r/Firearms":10, 'r/Conservative':1}

In [66]:
def convert_labels(row):
    row['subreddit'] = label_mapping.get(row['subreddit'], -1)  # Use -1 as a default if subreddit not in mapping
    return row

In [67]:
node_data = node_data.apply(convert_labels, axis=1)

In [68]:
node_data['subreddit'] = node_data['subreddit'].astype(int)

In [14]:
from torch_geometric.data import Data

In [69]:
x = torch.ones((len(user_ids), 1), dtype=torch.float)

y = torch.tensor(node_data['subreddit'].values, dtype=torch.long)

In [56]:
df["subreddit"].value_counts()

subreddit
r/politics                2947
r/Conservative             502
r/NoNewNormal              131
r/Coronavirus               91
r/LockdownSkepticism        78
r/JoeBiden                  68
r/MensRights                66
r/DebateVaccines            30
r/climateskeptics           27
r/Firearms                  25
r/conservatives             19
r/progun                    19
r/climatechange             14
r/liberalgunowners          13
r/Republican                11
r/CoronavirusUK             11
r/Abortiondebate             8
r/democrats                  8
r/prolife                    8
r/uspolitics                 8
r/vaxxhappened               7
r/LockdownCriticalLeft       6
r/Impeach_Trump              5
r/GunsAreCool                4
r/feminisms                  4
r/CoronavirusUS              4
r/TrueAntiVaccination        4
r/prochoice                  4
r/ConservativesOnly          3
r/DebateVaccine              3
r/AntiVaxxers                3
r/Masks4All                  

In [70]:
node_data['subreddit'].value_counts()

subreddit
 0     2947
 1      502
-1      185
 2      131
 4       91
 5       78
 6       68
 7       66
 8       30
 9       27
 10      25
Name: count, dtype: int64

In [71]:
data = Data(x=x, edge_index=edge_index, y=y)

In [73]:
torch.save(data, 'graph_data_percentage2.pt')